# Obstructive Sleep Apnea Syndrome (OSAS) classification
We use a publicly available dataset - [OSASUD](https://www.nature.com/articles/s41597-022-01272-y) (available [here](https://figshare.com/collections/A_dataset_of_stroke_unit_recordings_for_the_detection_of_Obstructive_Sleep_Apnea_Syndrome/5630890) to train a model to detect whether a patient is healthy or has any of the OSAS subtypes - HYPOPNEA, APNEA-CENTRAL, APNEA-OBSTRUCTIVE, APNEA-MIXED.
Dataset reference-
Bernardini A, Brunello A, Gigli GL, Montanari A, Saccomanno N. OSASUD: A dataset of stroke unit recordings for the detection of Obstructive Sleep Apnea Syndrome. Scientific Data. 2022 Apr;9(1):177. DOI: 10.1038/s41597-022-01272-y. PMID: 35440646; PMCID: PMC9018698.

## Data Analysis
We first load and analyze the data

### Imports

In [ ]:
# Necessary imports
import pandas as pd
import numpy as np
import pickle
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt

### Data loading
The data is a pandas dataframe stored in 'data' folder as a _.pickle_ file. We load it and display the first few rows and the datatypes of each column

In [ ]:
# Path to the dataset (assuming it is in the same directory as the notebook)
path = './data/dataset_OSAS.pickle'

# Loading the dataset
with open(path, 'rb') as file:
    dataset = pickle.load(file)

display(dataset.head(40))

display(dataset.dtypes)

### Number of classes
We display the class names that the model needs to classify

In [ ]:
class_names = list(dataset['event'].unique())

As we can see from above, this is a five class classification problem.

Now we display some basic information about the dataset

In [ ]:
print("Number of rows:", dataset.shape[0])
print("Number of columns:", dataset.shape[1])
print("Number of distinct patients:", len(dataset['patient'].unique()))

### Create a summary of dataframe
We create a summary about the dataframe containing the number of recorded hours, number of apnea, hypopnea events and information about null/missing values/

In [ ]:
# Function that, given an array of boolean values, outputs the "begin_index" and "end_index" of each contiguous block of TRUEs
def one_runs(a):
    iszero = np.concatenate(([0], np.equal(a, 1).astype(np.int8), [0]))
    absdiff = np.abs(np.diff(iszero))
    ranges = np.where(absdiff == 1)[0].reshape(-1, 2)
    return ranges

# Calculate validation data for each patient
validation_pandas = []
pbar = tqdm(desc="Processed patients", total=len(dataset['patient'].unique()))
for pat in np.unique(dataset['patient'])[np.argsort(np.unique(dataset['patient']).astype(np.int8))]:
    temp = dataset[dataset['patient'] == pat]

    tmp_null_pleth = np.asarray([np.isnan(x) for x in temp['signal_pleth']]).flatten()
    tmp_null_ecg_i = np.asarray([np.isnan(x) for x in temp['signal_ecg_i']]).flatten()
    tmp_null_ecg_ii = np.asarray([np.isnan(x) for x in temp['signal_ecg_ii']]).flatten()
    tmp_null_ecg_iii = np.asarray([np.isnan(x) for x in temp['signal_ecg_iii']]).flatten()

    pandas_row = [pat, # patient ID
                  round(len(temp) / 3600, 1), # recording duration (hours)
                  round(len(one_runs(temp['anomaly'].values)) / (len(temp) / 3600), 1), # AHI
                  len(one_runs(temp[temp['event'] != 'HYPOPNEA']['anomaly'])), # number of apnea events
                  len(one_runs(temp[(temp['event'] != 'APNEA-CENTRAL') & (temp['event'] != 'APNEA-MIXED') & (temp['event'] != 'APNEA-OBSTRUCTIVE')]['anomaly'])), # number of hypopnea events
                  round(np.mean([x[1] - x[0] + 1 for x in one_runs(temp['anomaly'].values)])), # average duration of (hypo)apnea events (seconds)
                  round(np.std([x[1] - x[0] + 1 for x in one_runs(temp['anomaly'].values)])), # standard deviation of the duration of (hypo)apnea events (seconds)
                  round(100 * np.sum(np.isnan(temp['HR(bpm)'])) / len(temp), 1), # percentage of null HR values
                  round(100 * np.sum(np.isnan(temp['SpO2(%)'])) / len(temp), 1), # percentage of null SpO2 values
                  round(100 * np.sum(np.isnan(temp['PI(%)'])) / len(temp), 1), # percentage of null PI values
                  round(100 * np.sum(np.isnan(temp['RR(rpm)'])) / len(temp), 1), # percentage of null RR values
                  round(100 * np.sum(np.isnan(temp['PVCs(/min)'])) / len(temp), 1), # percentage of null PVC values
                  round(100 * np.sum(tmp_null_pleth) / len(tmp_null_pleth), 1), # percentage of null pleth values,
                  round(100 * np.sum(tmp_null_ecg_i) / len(tmp_null_ecg_i), 1), # percentage of null ecg i values,
                  round(100 * np.sum(tmp_null_ecg_ii) / len(tmp_null_ecg_ii), 1), # percentage of null ecg ii values,
                  round(100 * np.sum(tmp_null_ecg_iii) / len(tmp_null_ecg_iii), 1), # percentage of null ecg iii values
                 ]

    validation_pandas.append(pandas_row)

    pbar.update(1)


validation_pandas = pd.DataFrame(validation_pandas, columns=['patient',
                                                             'recording duration (hrs)',
                                                             'AHI',
                                                             '# apnea events',
                                                             '# hypopnea events',
                                                             'avg duration (hypo)apnea events',
                                                             'stddev duration (hypo)apnea events',
                                                             '% null HR',
                                                             '% null SpO2',
                                                             '% null PI',
                                                             '% null RR',
                                                             '% null PVC',
                                                             '% null pleth',
                                                             '% null ecg i',
                                                             '% null ecg ii',
                                                             '% null ecg iii'
                                                            ])

display(validation_pandas)

# Save the dataframe to a csv file
# validation_pandas.to_csv('patient_data.csv')

### Patient wise box plot of features

In [ ]:
# Boxplots of the ECG and PPG derived data
plt.figure(figsize=(20, 20))
pbar = tqdm(desc="Processed features", total=5)
for i, column in enumerate(['HR(bpm)', 'SpO2(%)', 'PI(%)', 'RR(rpm)', 'PVCs(/min)']):
    plt.subplot(5, 1, i+1)
    plot_data = []
    for pat in np.unique(dataset['patient'])[np.argsort(np.unique(dataset['patient']).astype(np.int8))]:
        temp = dataset[dataset['patient'] == pat][column]
        plot_data.append([x for x in list(temp.values) if not np.isnan(x)])
    plt.boxplot(plot_data)
    if i == 4:
        plt.xlabel("Patient ID")
    plt.ylabel(column)
    pbar.update(1)
pbar.close()
plt.show()

### Patient wise re-assembly
We create a python dictionary that has the patient id as keys and all the features are encoded as another dictionary which is the value of patient key.

In [ ]:
# The following code re-assembles the time series related to each patient
patient_map_features = {} # given a patient, the map returns a map that, given feature, returns its whole time series
pbar = tqdm(desc="Processed patients", total=len(dataset['patient'].unique()))
for pat in dataset['patient'].unique():
    temp = dataset[dataset['patient'] == pat]
    feature_map_ts = {}
    for col in dataset.columns[1:]:
        if 'signal' not in col and 'PSG_' not in col:
            feature_map_ts[col] = temp[col].values
        else:
            feature_map_ts[col] = np.concatenate(temp[col].values)
    patient_map_features[pat] = feature_map_ts
    pbar.update(1)
pbar.close()

### Example plots using the above dictionary
We can now plot any feature value by giving the patient id and column name. We can also control how much information to show by slicing the data upto our required number of seconds.
For example-
`patient_map_features['15']['PI(%)'][:20000]` shows the first **20000** seconds of feature **PI(%)** for patient **15**

#### Perfusion Index (PI)

In [ ]:
# PI time series of patient 1. Observe that there are some outlier values that might be usefult correcting, for example using a weighting average approach.
plt.figure(figsize=(20, 6))
plt.title('Perfusion Index')
plt.plot(patient_map_features['15']['PI(%)'][:20000])
plt.show()

#### SpO2 signal
We show another example - first 2000 seconds of SpO2 of patient 10

In [ ]:
plt.figure(figsize=(20, 6))
plt.title('SpO2')
plt.plot(patient_map_features['10']['SpO2(%)'][:2000])
plt.show()

#### PSG Abdomen signal
First 2000 seconds of PSG_Abdomen of patient 2

In [ ]:
# Outliers produced by movements in patient 10's PSG_Abdomen time series
plt.figure(figsize=(20, 6))
plt.title('PSG Abdomen')
plt.plot(patient_map_features['2']['PSG_Abdomen'][:2000])
plt.show()

#### ECG Signal 2

In [ ]:
plt.figure(figsize=(20, 6))
plt.plot(patient_map_features['3']['signal_ecg_ii'][:2000])
plt.show()

## PSG Flow

In [ ]:
plt.figure(figsize=(20, 6))
plt.plot(patient_map_features['3']['PSG_Flow'][:2000])
plt.show()